In [1]:
!git clone https://github.com/karpathy/nanoGPT.git

Cloning into 'nanoGPT'...
remote: Enumerating objects: 689, done.
remote: Total 689 (delta 0), reused 0 (delta 0), pack-reused 689 (from 1)
Receiving objects: 100% (689/689), 975.24 KiB | 19.50 MiB/s, done.
Resolving deltas: 100% (382/382), done.


In [ ]:
%%bash
cd /kaggle/working/nanoGPT
mkdir -p data/abc

ln -s /kaggle/input/training-data/train.bin data/abc/train.bin
ln -s /kaggle/input/validation-set/val.bin  data/abc/val.bin
ln -s /kaggle/input/meta-data/meta.pkl      data/abc/meta.pkl


**Small Transformer** 

In [10]:
%%bash
cd /kaggle/working/nanoGPT
cat << 'EOF' > config/abc_small.py
# config/abc_small.py
# Small GPT (~5–6M parameters)

# -------------------
# Output & logging
# -------------------
out_dir = '/kaggle/working/nanoGPT_runs/out-abc-small'
eval_interval = 1000
eval_iters = 200
log_interval = 200
always_save_checkpoint = True

# -------------------
# Dataset
# -------------------
dataset = 'abc'

# -------------------
# Batch / streaming
# -------------------
block_size = 256
batch_size = 8
gradient_accumulation_steps = 16
# Effective tokens / iteration = 32768

# -------------------
# Model (Small)
# -------------------
n_layer = 8
n_head  = 8
n_embd  = 256
dropout = 0.1

# -------------------
# Training (1 epoch = 200M tokens)
# -------------------
learning_rate = 3e-4
max_iters = 6104
lr_decay_iters = max_iters
warmup_iters = 500
min_lr = 1e-5

weight_decay = 1e-1
beta1 = 0.9
beta2 = 0.95
grad_clip = 1.0

# -------------------
# System
# -------------------
device = 'cuda'
dtype = 'float16'
compile = False
EOF


In [12]:
%%bash
cd /kaggle/working/nanoGPT
python -u train.py config/abc_small.py | tee small.log




Overriding config with config/abc_small.py:
# config/abc_small.py
# Small GPT (~5–6M parameters)

# -------------------
# Output & logging
# -------------------
out_dir = '/kaggle/working/nanoGPT_runs/out-abc-small'
eval_interval = 1000
eval_iters = 200
log_interval = 200
always_save_checkpoint = True

# -------------------
# Dataset
# -------------------
dataset = 'abc'

# -------------------
# Batch / streaming
# -------------------
block_size = 256
batch_size = 8
gradient_accumulation_steps = 16
# Effective tokens / iteration = 32768

# -------------------
# Model (Small)
# -------------------
n_layer = 8
n_head  = 8
n_embd  = 256
dropout = 0.1

# -------------------
# Training (1 epoch = 200M tokens)
# -------------------
learning_rate = 3e-4
max_iters = 6104
lr_decay_iters = max_iters
warmup_iters = 500
min_lr = 1e-5

weight_decay = 1e-1
beta1 = 0.9
beta2 = 0.95
grad_clip = 1.0

# -------------------
# System
# -------------------
device = 'cuda'
dtype = 'float16'
compile = False


/kaggle/working/nanoGPT/train.py:196: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(dtype == 'float16'))


**Medium Transformer**

In [14]:
%%bash
cd /kaggle/working/nanoGPT
cat << 'EOF' > config/abc_medium.py
# config/abc_medium.py
# Medium GPT (~20M parameters)

# -------------------
# Output & logging
# -------------------
out_dir = '/kaggle/working/nanoGPT_runs/out-abc-medium'
eval_interval = 1000
eval_iters = 200
log_interval = 200
always_save_checkpoint = True

# -------------------
# Dataset
# -------------------
dataset = 'abc'

# -------------------
# Batch / streaming
# -------------------
block_size = 256
batch_size = 8
gradient_accumulation_steps = 16
# Effective tokens / iteration = 32768

# -------------------
# Model (Medium)
# -------------------
n_layer = 12
n_head  = 6
n_embd  = 384
dropout = 0.1

# -------------------
# Training (1 epoch = 200M tokens)
# -------------------
learning_rate = 3e-4
max_iters = 6104
lr_decay_iters = max_iters
warmup_iters = 500
min_lr = 1e-5

weight_decay = 1e-1
beta1 = 0.9
beta2 = 0.95
grad_clip = 1.0

# -------------------
# System
# -------------------
device = 'cuda'
dtype = 'float16'
compile = False
EOF


In [15]:
%%bash
cd /kaggle/working/nanoGPT
python -u train.py config/abc_medium.py | tee medium.log


Overriding config with config/abc_medium.py:
# config/abc_medium.py
# Medium GPT (~20M parameters)

# -------------------
# Output & logging
# -------------------
out_dir = '/kaggle/working/nanoGPT_runs/out-abc-medium'
eval_interval = 1000
eval_iters = 200
log_interval = 200
always_save_checkpoint = True

# -------------------
# Dataset
# -------------------
dataset = 'abc'

# -------------------
# Batch / streaming
# -------------------
block_size = 256
batch_size = 8
gradient_accumulation_steps = 16
# Effective tokens / iteration = 32768

# -------------------
# Model (Medium)
# -------------------
n_layer = 12
n_head  = 6
n_embd  = 384
dropout = 0.1

# -------------------
# Training (1 epoch = 200M tokens)
# -------------------
learning_rate = 3e-4
max_iters = 6104
lr_decay_iters = max_iters
warmup_iters = 500
min_lr = 1e-5

weight_decay = 1e-1
beta1 = 0.9
beta2 = 0.95
grad_clip = 1.0

# -------------------
# System
# -------------------
device = 'cuda'
dtype = 'float16'
compile = F

/kaggle/working/nanoGPT/train.py:196: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(dtype == 'float16'))


RNN Training- Different Sizes

In [22]:
%%bash
cd /kaggle/working/nanoGPT
cat << 'EOF' > lstm_scaling.py
import os, time, math, json, pickle
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

# =========================
# CONFIG (MATCH TRANSFORMER)
# =========================
DATA_DIR = os.path.join("data", "abc")
OUT_DIR  = "out-lstm"
DEVICE   = "cuda" if torch.cuda.is_available() else "cpu"

BATCH_SIZE = 8
BLOCK_SIZE = 256
GRAD_ACCUM = 16                 # 8*256*16 = 32768 tokens/step
TOKEN_BUDGET = 200_000_000      # <<< 200M TOKENS (FINAL)

LR = 3e-4
MIN_LR = 1e-5
WARMUP_ITERS = 500
WEIGHT_DECAY = 1e-1
BETAS = (0.9, 0.95)
GRAD_CLIP = 1.0

EVAL_INTERVAL = 2000
EVAL_ITERS = 200
LOG_INTERVAL = 500

DTYPE = torch.bfloat16 if DEVICE == "cuda" else torch.float32

# =========================
# DATA
# =========================
train_data = np.memmap(os.path.join(DATA_DIR, "train.bin"), dtype=np.uint16, mode="r")
val_data   = np.memmap(os.path.join(DATA_DIR, "val.bin"),   dtype=np.uint16, mode="r")

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(0, len(data) - BLOCK_SIZE - 1, (BATCH_SIZE,))
    x = torch.stack([torch.from_numpy((data[i:i+BLOCK_SIZE]).astype(np.int64)) for i in ix])
    y = torch.stack([torch.from_numpy((data[i+1:i+1+BLOCK_SIZE]).astype(np.int64)) for i in ix])
    return x.to(DEVICE), y.to(DEVICE)

@torch.no_grad()
def estimate_loss(model):
    model.eval()
    losses = []
    for _ in range(EVAL_ITERS):
        x, y = get_batch("val")
        with torch.autocast(device_type="cuda", dtype=DTYPE, enabled=(DEVICE=="cuda")):
            _, loss = model(x, y)
        losses.append(loss.item())
    model.train()
    return float(np.mean(losses))

# =========================
# MODEL
# =========================
class LSTMLM(nn.Module):
    def __init__(self, vocab_size, n_embd, n_layer):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, n_embd)
        self.lstm = nn.LSTM(n_embd, n_embd, n_layer, batch_first=True)
        self.ln = nn.LayerNorm(n_embd)
        self.head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        x = self.emb(idx)
        y, _ = self.lstm(x)
        y = self.ln(y)
        logits = self.head(y)
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
        return logits, loss

def count_params(m):
    return sum(p.numel() for p in m.parameters())

def lr_schedule(it, max_iters):
    if it < WARMUP_ITERS:
        return LR * it / WARMUP_ITERS
    if it >= max_iters:
        return MIN_LR
    decay_ratio = (it - WARMUP_ITERS) / (max_iters - WARMUP_ITERS)
    return MIN_LR + 0.5 * (LR - MIN_LR) * (1 + math.cos(math.pi * decay_ratio))

# =========================
# TRAIN ONE MODEL
# =========================
def train_model(name, n_layer, n_embd):
    with open(os.path.join(DATA_DIR, "meta.pkl"), "rb") as f:
        vocab_size = pickle.load(f)["vocab_size"]

    os.makedirs(OUT_DIR, exist_ok=True)
    log_path = os.path.join(OUT_DIR, f"{name}.jsonl")

    model = LSTMLM(vocab_size, n_embd, n_layer).to(DEVICE)
    n_params = count_params(model)

    tokens_per_iter = BATCH_SIZE * BLOCK_SIZE * GRAD_ACCUM
    max_iters = math.ceil(TOKEN_BUDGET / tokens_per_iter)

    opt = torch.optim.AdamW(model.parameters(), lr=LR, betas=BETAS, weight_decay=WEIGHT_DECAY)

    print(f"\n{name}: {n_params/1e6:.2f}M params | iters={max_iters}")
    start = time.time()

    with open(log_path, "w") as f:
        f.write(json.dumps({
            "name": name,
            "params": n_params,
            "token_budget": TOKEN_BUDGET,
            "tokens_per_iter": tokens_per_iter,
            "max_iters": max_iters
        }) + "\n")

    for it in range(1, max_iters + 1):
        lr = lr_schedule(it, max_iters)
        for pg in opt.param_groups:
            pg["lr"] = lr

        opt.zero_grad(set_to_none=True)
        total_loss = 0.0

        for _ in range(GRAD_ACCUM):
            xb, yb = get_batch("train")
            with torch.autocast(device_type="cuda", dtype=DTYPE, enabled=(DEVICE=="cuda")):
                _, loss = model(xb, yb)
                loss = loss / GRAD_ACCUM
            loss.backward()
            total_loss += loss.item()

        torch.nn.utils.clip_grad_norm_(model.parameters(), GRAD_CLIP)
        opt.step()

        if it % LOG_INTERVAL == 0:
            print(f"{name} | iter {it}/{max_iters}")

        if it % EVAL_INTERVAL == 0 or it == max_iters:
            val_loss = estimate_loss(model)
            with open(log_path, "a") as f:
                f.write(json.dumps({"iter": it, "val_loss": val_loss}) + "\n")
            print(f"{name} | VAL @ {it}: {val_loss:.4f}")

    print(f"{name} DONE in {(time.time()-start)/3600:.2f}h")

# =========================
# RUN 4 SIZES
# =========================
if __name__ == "__main__":
    runs = [
        ("lstm_tiny",   2,  256),
        ("lstm_small",  3,  448),
        ("lstm_medium", 3,  896),
        ("lstm_large",  4, 1280),
    ]
    for r in runs:
        train_model(*r)
EOF


In [23]:
%%bash
cd /kaggle/working/nanoGPT
python -u lstm_scaling.py | tee lstm_scaling.log


lstm_tiny: 1.10M params | iters=6104
lstm_tiny | iter 500/6104
lstm_tiny | iter 1000/6104
lstm_tiny | iter 1500/6104
lstm_tiny | iter 2000/6104
lstm_tiny | VAL @ 2000: 0.7188
lstm_tiny | iter 2500/6104
lstm_tiny | iter 3000/6104
lstm_tiny | iter 3500/6104
lstm_tiny | iter 4000/6104
lstm_tiny | VAL @ 4000: 0.6507
lstm_tiny | iter 4500/6104
lstm_tiny | iter 5000/6104
lstm_tiny | iter 5500/6104
lstm_tiny | iter 6000/6104
lstm_tiny | VAL @ 6000: 0.6413
lstm_tiny | VAL @ 6104: 0.6497
lstm_tiny DONE in 0.72h

lstm_small: 4.92M params | iters=6104
lstm_small | iter 500/6104
lstm_small | iter 1000/6104
lstm_small | iter 1500/6104
lstm_small | iter 2000/6104
lstm_small | VAL @ 2000: 0.6475
lstm_small | iter 2500/6104
lstm_small | iter 3000/6104
lstm_small | iter 3500/6104
lstm_small | iter 4000/6104
lstm_small | VAL @ 4000: 0.5688
lstm_small | iter 4500/6104
lstm_small | iter 5000/6104
lstm_small | iter 5500/6104
lstm_small | iter 6000/6104
lstm_small | VAL @ 6000: 0.5453
lstm_small | VAL @ 61